In [54]:
from typing import Any

import concrete_db
import concrete.operators as ops
from concrete.orchestrators import SoftwareOrchestrator
from concrete.projects.dag_project import DAGNode, Project

In [29]:
promptEngineer = ops.PromptEngineer()

In [15]:
prompt = """
I'm an AI engineer. I'm trying to design a demo system to demonstrate the power of an multi-agentic architecture
in making a sports streaming platform. Provide me a prompt for a Marketer, a Salesperson, and a Designer.
"""
print(promptEngineer.chat(prompt).text)

### Marketer Prompt:
"As a marketing expert for a multi-agent sports streaming platform, your task is to develop a comprehensive marketing strategy that highlights the unique features and benefits of our platform. Consider the target audience, competitive landscape, and potential partnerships. Create a campaign that includes social media outreach, influencer collaborations, and promotional events. Ensure your strategy is adaptable to different sports and regions, and encourage innovative ideas that can capture the attention of sports fans."

### Salesperson Prompt:
"As a salesperson for our multi-agent sports streaming platform, your goal is to create a persuasive sales pitch that effectively communicates the value proposition of our service to potential clients. Focus on the unique selling points, such as personalized content, real-time analytics, and community engagement features. Prepare a presentation that addresses common objections and highlights success stories or testimonials. 

In [58]:
class Marketer(ops.Operator):
    instructions = """
As a marketing expert for a multi-agent sports streaming platform,
your task is to develop a comprehensive marketing strategy that highlights the
unique features and benefits of our platform.
Consider the target audience, competitive landscape, and potential partnerships.
Create a campaign that includes social media outreach, influencer collaborations,
and promotional events. Ensure your strategy is adaptable to different sports and regions,
and encourage innovative ideas that can capture the attention of sports fans.
"""

    def segment_audience(
        self,
        product_idea: str,
        options: dict[str, Any] = {},
    ) -> str:
        return f"""Given the following product idea:

        START PRODUCT IDEA
        {product_idea}
        END PRODUCT IDEA

        Develop profiles for different segments of the target audience,
        including demographics, interests, and viewing habits.
        This will help tailor marketing messages and campaigns to resonate with specific groups.
        Based on the profiles, provide feedback on the product idea.
        """


class SalesPerson(ops.Salesperson):
    instructions = ops.Salesperson.instructions + "\n" + """
As a salesperson for our multi-agent sports streaming platform,
your goal is to create a persuasive sales pitch that effectively communicates
the value proposition of our service to potential clients. Focus on the unique selling points,
such as personalized content, real-time analytics, and community engagement features.
Prepare a presentation that addresses common objections and highlights success stories or testimonials.
Be specific about the benefits for different customer segments, and think creatively about
how to close deals in a competitive market.
"""

class Designer(ops.Designer):
    instructions = ops.Designer.instructions + "\n" + """
As a designer for our multi-agent sports streaming platform,
your mission is to create an engaging and user-friendly interface that enhances the viewer experience.
Consider the needs of diverse users, including casual viewers and hardcore fans.
Develop wireframes and mockups that showcase innovative features like interactive stats,
customizable layouts, and social sharing options.
Ensure your designs are adaptable for various devices and screen sizes,
and encourage creative solutions that make the platform visually appealing and easy to navigate.
"""

    def revise_design(
        self,
        design: str,
        feedback: str,
        options: dict[str, Any] = {},
    ) -> str:
        return f"""Given the following design:

        START DESIGN
        {design}
        END DESIGN

        Enhance the original concept while addressing the points raised in the following feedback:

        START FEEDBACK
        {feedback}
        END FEEDBACK

1. **Clarity**: Ensure you fully understand the feedback provided.
2. **Completeness**: Consider all aspects of the design, including aesthetics, functionality, and user experience, as you incorporate the feedback.
3. **Specificity**: Focus on the specific elements mentioned in the feedback. For example, if the feedback suggests changing the color scheme, provide a few alternative palettes that align with the overall theme.
4. **Adaptability**: Be open to making broader changes if necessary. If the feedback indicates a fundamental issue with the design, be prepared to rethink your approach while still maintaining the core concept.
5. **Creativity**: Explore innovative solutions that not only address the feedback but also enhance the design. Think outside the box and consider how you can elevate the design beyond the initial idea.

Now, revise the design idea accordingly, ensuring that you effectively integrate the feedback while showcasing your creativity and design expertise.
"""

In [31]:
promptBoostPrompt = f"""
I want to augment my prompts to a Marketer LLM agent whose instructions are as follows:
{Marketer.instructions}
Can you provide extra functionalities for said agent? 
"""
print(promptEngineer.chat(promptBoostPrompt).text)

1. **Audience Segmentation**: Develop detailed profiles for different segments of the target audience, including demographics, interests, and viewing habits. This will help tailor marketing messages and campaigns to resonate with specific groups.

2. **Competitive Analysis**: Conduct a thorough analysis of competitors in the sports streaming market. Identify their strengths, weaknesses, and unique selling propositions (USPs) to inform your strategy and differentiate your platform.

3. **Content Creation**: Generate engaging content ideas for social media, blogs, and promotional materials that highlight the platform's unique features. This could include video highlights, behind-the-scenes content, and user testimonials.

4. **Performance Metrics**: Establish key performance indicators (KPIs) to measure the success of marketing campaigns. This includes tracking engagement rates, conversion rates, and audience growth across different channels.

5. **Partnership Development**: Identify pot

In [57]:
designPromptBoostPrompt = f"""
Instruct a designer to revise a provided design idea given a set of feedback.
"""

designPromptBoost = promptEngineer.chat(designPromptBoostPrompt).text

print(designPromptBoost)

You are a skilled designer tasked with revising a design idea based on specific feedback. Your goal is to enhance the original concept while addressing the points raised in the feedback. 

1. **Clarity**: Ensure you fully understand the feedback provided. If any points are unclear, seek clarification before proceeding.
2. **Completeness**: Consider all aspects of the design, including aesthetics, functionality, and user experience, as you incorporate the feedback.
3. **Specificity**: Focus on the specific elements mentioned in the feedback. For example, if the feedback suggests changing the color scheme, provide a few alternative palettes that align with the overall theme.
4. **Adaptability**: Be open to making broader changes if necessary. If the feedback indicates a fundamental issue with the design, be prepared to rethink your approach while still maintaining the core concept.
5. **Creativity**: Explore innovative solutions that not only address the feedback but also enhance the des

In [37]:
design_prompt = promptEngineer.chat("Give me a prompt to tell a designer to come up with an inuitive sports streaming platform user experience.").text
print(design_prompt)

You are a UX designer tasked with creating an intuitive user experience for a sports streaming platform. Your goal is to design a seamless interface that enhances user engagement and accessibility. Consider the following aspects:

1. **User Journey**: Map out the typical user journey from account creation to watching a live event. Identify key touchpoints and potential pain points.
2. **Navigation**: Develop a clear and straightforward navigation structure that allows users to easily find live games, highlights, and on-demand content.
3. **Personalization**: Incorporate features that allow users to personalize their experience, such as favorite teams, notifications for upcoming games, and tailored content recommendations.
4. **Visual Design**: Create a visually appealing layout that reflects the excitement of sports while ensuring readability and ease of use.
5. **Accessibility**: Ensure the platform is accessible to all users, including those with disabilities. Consider features like 

In [ ]:
count = 1

project = Project()

designers = [DAGNode(
    "chat",
    Designer(),
    default_task_kwargs={
        "message": design_prompt,
    },
) for _ in range(count)]

marketers = [DAGNode(
    "segment_audience",
    Marketer(),
) for _ in range(count)]

revisers = [DAGNode(
    "revise_design",
    Designer(),
) for _ in range(count)]

for i in range(count):
    project.add_node(designers[i])
    project.add_node(marketers[i])
    project.add_node(revisers[i])

    project.add_edge(marketers[i], designers[i], "product_idea")
    project.add_edge(revisers[i], designers[i], "design")
    project.add_edge(revisers[i], marketers[i], "feedback")


operator_response = []
async for operator, response in project.execute():
    operator_response.append([operator, response.text])
    print(f"{operator_response[-1][0]}\n{operator_response[-1][1]}\n")

Designer
### User Journey Mapping
1. **Account Creation**: 
   - **Touchpoints**: Sign-up page, email verification, profile setup.
   - **Pain Points**: Complicated sign-up forms, lack of clear instructions.

2. **Home Screen**: 
   - **Touchpoints**: Featured live events, personalized recommendations, quick access to favorite teams.
   - **Pain Points**: Overwhelming information, difficulty finding specific content.

3. **Content Discovery**: 
   - **Touchpoints**: Navigation bar, search functionality, filters for live games, highlights, and on-demand content.
   - **Pain Points**: Inefficient search results, lack of intuitive filtering options.

4. **Watching Live Events**: 
   - **Touchpoints**: Video player interface, interactive stats, social sharing options.
   - **Pain Points**: Cluttered video player, difficulty accessing stats or sharing content.

5. **Post-Event Engagement**: 
   - **Touchpoints**: Highlights, post-game analysis, notifications for upcoming games.
   - **Pain 

In [ ]:
designer_output = operator_response[-1][1]
software_orchestrator = SoftwareOrchestrator()
async for operator, response in software_orchestrator.process_new_project(designer_output):
    print(operator)
    print(response)
    print()

Executive
{
    "components": [
        "Create User Model",
        "Implement User Registration API",
        "Implement Email Verification Service",
        "Create User Profile Setup Component",
        "Design Home Screen Layout",
        "Implement Featured Live Events Component",
        "Implement Personalized Recommendations Algorithm",
        "Create Navigation Bar Component",
        "Implement Search Functionality",
        "Create Filters for Content Discovery",
        "Design Video Player Interface",
        "Implement Interactive Stats Display",
        "Integrate Social Sharing Options",
        "Create Post-Event Highlights Component",
        "Implement Notifications System",
        "Design Settings Page Layout",
        "Implement Favorite Teams Selection Feature",
        "Implement Push Notification Service",
        "Create Feedback Button Component",
        "Implement Survey Pop-Up Mechanism",
        "Design Community Forum Layout",
        "Create Home Scre